Found GB data. Need to develop workflow to iteratively grab other team's data.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

import matplotlib.pyplot as plt

import pickle

## Goal
Pull Elo data from dynamic graph on https://projects.fivethirtyeight.com/complete-history-of-the-nfl/#gb.

### JS source data

I found data behind graph by exploring 'Insecpt Page' tabs, per instructions on: https://onlinejournalismblog.com/2017/05/10/how-to-find-data-behind-chart-map-using-inspector/

Selenium & BS didn't work on below url, so just used Regex

In [2]:
'''
chromedriver = "/Users/kendra/ds/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

url_data = 'https://projects.fivethirtyeight.com/complete-history-of-the-nfl/js/bundle.js?v=77c0dce50abab847428693938063b9db'
response = requests.get(url_data)
page = response.text

with open('elo_page_text.pkl', 'wb') as picklefile:
    pickle.dump(page, picklefile)
'''


with open('elo_page_text.pkl','rb') as picklefile:
    page = pickle.load(picklefile)

In [3]:
page[:500]

'!function t(e,n,y){function r(i,o){if(!n[i]){if(!e[i]){var a="function"==typeof require&&require;if(!o&&a)return a(i,!0);if(x)return x(i,!0);var s=new Error("Cannot find module \'"+i+"\'");throw s.code="MODULE_NOT_FOUND",s}var u=n[i]={exports:{}};e[i][0].call(u.exports,function(t){var n=e[i][1][t];return r(n?n:t)},u,u.exports,t,e,n,y)}return n[i].exports}for(var x="function"==typeof require&&require,i=0;i<y.length;i++)r(y[i]);return r}({1:[function(t,e,n){function y(t,e,n,y){return!1!==e&&null!=e&'

### Get list of team abbreviations

In [4]:
r2 = re.compile(r'({name:)')
itr2 = re.finditer(r2, page)
next_list = [[m.start(0), m.end(0)] for m in itr2]
next_list = next_list[4:]

r3 = re.compile(r'\"(.*?)\"')
team_abbrevs = [re.search(r3, page[n[0]:n[0]+20]).group(1) for n in next_list]

In [5]:
len(team_abbrevs)

53

In [19]:
team_abbrevs[-1]

'Hou'

### Pull out data for each team

In [72]:
master_df = pd.DataFrame()

In [73]:
err_log = []
for ta in team_abbrevs:
#     print(ta)
    try:
        df = pd.DataFrame(columns=['Year','Elo','Team'])
        re_str = '"' + ta + '"'
        re_str2 ="%r"%re_str
        r1 = re.compile(re_str2[1:-1])
        itr = re.finditer(r1, page)
        tm_list = [[m.start(0), m.end(0)] for m in itr]
        tm_text = page[tm_list[0][0]:]

        r2 = re.compile(r'({name:)')
        tm_end = re.search(r2, tm_text).start(0)
        tm_data = tm_text[11:tm_end-3].split('}')
        tm_data2 = [x.strip('{,').split(',') for x in tm_data][1:-1]
        tm_data3 = [[float(i[0].split(':')[1]), float(i[1].split(':')[1]), ta] for i in tm_data2]
        df = pd.DataFrame(tm_data3)
        master_df = pd.concat([master_df, df], ignore_index=True)
    except ValueError:
        print('Something went wrong for ' + ta)
        err_log.append(ta)
    except AttributeError:
        print ('Who the hell knows ' + ta)
#     xs, ys = list(zip(*tm_data3))
    

Who the hell knows Hou


In [76]:
master_df.columns = ['Year','Elo','Team']
master_df.sample(20)

,Year,Elo,Team
5962,1991.227,1554.040,Den
3305,1961.882,1462.360,Pit
8379,1998.000,1509.508,Min
3350,1972.825,1649.770,Pit
2127,2001.636,1484.030,Nyg
9624,1989.000,1352.240,Tb
8531,1976.333,1531.660,Mia
1811,1939.000,1628.420,Nyg
3671,1958.429,1388.580,Phi
8003,1980.667,1666.810,Dal


In [18]:
len(master_df['Team'].unique())

52

In [77]:
ta = 'Hou'
re_str = '"' + ta + '"'
re_str2 = "%r"%re_str

In [78]:
r1 = re.compile(re_str2[1:-1])
itr = re.finditer(r1, page)
tm_list = [[m.start(0), m.end(0)] for m in itr]

tm_text = page[tm_list[0][0]:]

r2 = re.compile(r'(function)')
tm_end = re.search(r2, tm_text).start(0)
tm_data = tm_text[11:tm_end-3].split('}')
tm_data2 = [x.strip('{,').split(',') for x in tm_data][:-4]

In [81]:
tm_data2[0]

[':[{x:2002', 'y:1300', 'p:"f"']

In [84]:
tm_data2[0][0] = tm_data2[0][0][3:]

In [86]:
tm_data2[0]

['x:2002', 'y:1300', 'p:"f"']

In [95]:
tm_data3 = [[float(i[0].split(':')[1]), float(i[1].split(':')[1]), ta] for i in tm_data2]
df = pd.DataFrame(tm_data3)
df.columns=['Year','Elo','Team']

In [96]:
df.head()

,Year,Elo,Team
0,2002.000,1300.00,Hou
1,2002.048,1327.80,Hou
2,2002.524,1228.30,Hou
3,2002.667,1309.16,Hou
4,2002.810,1278.64,Hou


In [94]:
master_df

,Year,Elo,Team
0,1920.231,1559.400,Rii
1,1920.308,1523.250,Rii
2,1920.462,1545.890,Rii
3,1920.582,1533.460,Rii
4,1920.615,1496.080,Rii
5,1920.769,1508.240,Rii
6,1921.000,1508.240,Rii
7,1921.242,1486.730,Rii
8,1921.538,1565.700,Rii
9,1921.615,1555.460,Rii


In [97]:
master_df = pd.concat([master_df, df], ignore_index=True)

In [99]:
master_df.sample(10)

,Year,Elo,Team
5256,1970.452,1648.870,Ind
3226,1945.385,1352.750,Pit
8105,2001.636,1417.030,Dal
121,1928.462,1557.130,Chi
7283,2011.182,1504.760,Buf
6135,1965.000,1513.224,Sd
6050,2008.136,1528.430,Den
10468,2017.364,1488.640,Hou
3783,1978.095,1436.590,Phi
8637,1996.545,1537.390,Mia


In [100]:
with open('elo_df', 'wb') as picklefile:
    pickle.dump(master_df, picklefile)

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(xs,ys);